<a href="https://colab.research.google.com/github/ckbjimmy/2019_tokyo/blob/master/causal_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Causal Inference Workshop

Hands-on Workshops by Satoshi Kimura / Wei-Hung Weng / Ryo Uchimido

March 7, 2019 @ TMDU


---
- [`doWhy` GitHub repo](https://github.com/Microsoft/dowhy)
- [`doWhy` document](https://causalinference.gitlab.io/dowhy/)
- Before running all cells, please go to `FILE > Save a copy in Drive` to save the colab notebook to your own Google Drive.

## Install dependencies


In [0]:
%%capture 
!git clone https://github.com/Microsoft/dowhy.git
!sudo apt install graphviz libgraphviz-dev graphviz-dev pkg-config
!pip install pygraphviz \
 --install-option="--include-path=/usr/include/graphviz" \
 --install-option="--library-path=/usr/lib/graphviz/"
!pip install -r ./dowhy/requirements.txt
!pip install --upgrade pandas
!pip install --upgrade statsmodels
!python ./dowhy/setup.py install
!wget -P ./dowhy/dowhy/causal_estimators/ \
https://raw.githubusercontent.com/ckbjimmy/2019_tokyo/master/logistic_regression_estimator.py

After running the installation cell, please go to `Runtime > Restart runtime` to reload the updated packages.

## Create a data simulator and define a graph

In [0]:
import numpy as np
import pandas as pd


class DataSimulator(object):
    def __init__(self):
        # Specify the model parameters
        self.sample_size = 5000

        # Specify the prevalence of A (exposure)
        self.p_A = 0.2

        # Parameters for the odds of S (selection)
        self.g0 = np.log(0.10/(1 - 0.10)) # log odds of S for ref group (A = 0 and U = 0)
        self.g1 = np.log(5.0) # log OR for effect of A on log odds of selection (OR = 5.0)
        self.g2 = np.log(5.0) # log OR for effect of U on log odds of selection (OR = 5.0)
        self.g3 = np.log(1.0)  # log OR for interaction between A and U on    (OR = 5.0)

        # Parameters for the odds of Y (outcome)
        # Is S = 0 for reference group correct?
        self.b0 = np.log(0.05/(1 - 0.05)) #log odds of Y for ref group (A = 0, U = 0, and S = 0)
        self.b1 = np.log(1.0) #log OR for effect of A on log odds of Y (OR = 1.0)
        self.b2 = np.log(5.0) #log OR for effect of U on log odds of Y (OR = 5.0)
    
    def prob_C(self, A, U):
        return np.exp(self.g0 + self.g1*A)/(1 + np.exp(self.g0 + self.g1*A))
  
    def prob_S(self, A, U):
        return np.exp(self.g0 + self.g1*A + self.g2*U + self.g3*U*A) / (1 + np.exp(self.g0 + self.g1*A + self.g2*U + self.g3*U*A))

    def prob_Y(self, A, U):
        return np.exp(self.b0 + self.b1*A +  self.b2*U) / (1 + np.exp(self.b0 + self.b1*A + self.b2*U))
      
    def get_graph(self, task):
        if task == 'confounder':
            g = '''
            graph [
            directed 1
            node[id "A" label "A"]
            node[id "Y" label "Y"]
            node[id "C" label "C"]
            edge[source "A" target "Y" weight 1]
            edge[source "C" target "A" weight 1]
            edge[source "C" target "Y" weight 1]
            ]
            '''

            d = '''
            digraph {
            A -> Y;
            C -> A;
            C -> Y;
            }
            '''

        elif task == 'collider':
            g = '''
            graph [
            directed 1
            node[id "A" label "A"]
            node[id "Y" label "Y"]
            node[id "C" label "C"]
            node[id "S" label "S"]
            edge[source "A" target "Y" weight 1]
            edge[source "A" target "S" weight 1]
            edge[source "Y" target "S" weight 1]
            edge[source "C" target "A" weight 1]
            edge[source "C" target "Y" weight 1]
            ]
            '''

            d = '''
            digraph {
            A -> Y;
            A -> S;
            C -> A;
            C -> Y;
            Y -> S;
            U[label="Unobserved Confounders"];
            U -> A;
            U -> Y;
            }
            '''
            
        return g, d

    def get_data(self, task):
        g, d = self.get_graph(task)
        ls_A = stats.binom.rvs(size=self.sample_size, n=1, p=self.p_A)
        ls_U = stats.norm.rvs(size=self.sample_size, loc=0, scale=1)
        p_S = [self.prob_S(ls_A[i], ls_U[i]) for i in range(self.sample_size)]
        p_Y = [self.prob_Y(ls_A[i], ls_U[i]) for i in range(self.sample_size)]
        
        if task == 'confounder':
            return pd.DataFrame({
                'A': ls_A, 
                'C': ls_U, 
                'prob_Y': p_Y, 
                'Y': stats.binom.rvs(size=self.sample_size, n=1, p=p_Y)
            }, columns=['A', 'C', 'prob_Y', 'Y']), g, d
          
        elif task == 'collider':
            return pd.DataFrame({
                'A': ls_A, 
                'C': ls_U, 
                'prob_S': p_S,
                'prob_Y': p_Y, 
                'S': stats.binom.rvs(size=self.sample_size, n=1, p=p_S),
                'Y': stats.binom.rvs(size=self.sample_size, n=1, p=p_Y)
            }, columns=['A', 'C', 'prob_S', 'prob_Y', 'S', 'Y']), g, d

## Load packages and inspect data

In [0]:
import sys
sys.path.append('dowhy')
import numpy as np
import pandas as pd
from scipy import stats
from IPython.display import Image, display

from dowhy.do_why import CausalModel
import dowhy.datasets
from dowhy.do_samplers.kernel_density_sampler import KernelDensitySampler
from dowhy.api.causal_data_frame import CausalDataFrame
from statsmodels.api import OLS

In [0]:
# # Using doWhy simulator
# data = dowhy.datasets.linear_dataset(
#     beta=10,
#     num_common_causes=5,
#     num_instruments=2,
#     num_samples=10000,
#     treatment_is_binary=True)

# data = dowhy.datasets.linear_dataset(beta=5,
#         num_common_causes=1,
#         num_instruments = 0,
#         num_samples=1000,
#         treatment_is_binary=True)
# data['dot_graph'] = 'digraph { v ->y;X0-> v;X0-> y;}'

# Simulate our own data

task = 'collider' # change to 'confounder' for another graph

sim = DataSimulator()
data = {}
data['df'], g, d = sim.get_data(task)
data['treatment_name'] = 'A'
data['outcome_name'] = 'Y'
data['gml_graph'] = g.replace('\n', '')
data['dot_graph'] = d.replace('\n', '')

In [0]:
data

### If we don't consider removing the collider (S)

In [0]:
df = data['df']
model = OLS(df['Y'], df[['C', 'A', 'S']])
result = model.fit()
result.summary()

### Considering the collider (S) using causal graph

In [0]:
# Create a causal model from the data and given graph.
model = CausalModel(
    data=data["df"],
    treatment=data["treatment_name"],
    outcome=data["outcome_name"],
    graph=data["gml_graph"])

model.view_model()
display(Image(filename="causal_model.png"))

In [0]:
# Identify causal effect and return target estimands
identified_estimand = model.identify_effect()
print(identified_estimand)

In [0]:
# Estimate the target estimand using a statistical method.
# estimate = model.estimate_effect(identified_estimand,
#                                  method_name="backdoor.propensity_score_matching")

estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.logistic_regression") # change to backdoor.linear_regression if the outcome is continuous

print(estimate)
print("Causal Estimate is " + str(estimate.value))

In [0]:
# # Refute the obtained estimate using multiple robustness checks.
refute_results = model.refute_estimate(identified_estimand, estimate,
                                       method_name="random_common_cause")

print(refute_results)

The estimated effect decrease from 0.0676 to 0.008.

## `test mcmc do sampler.ipynb`

In [0]:
df = data['df']
df['Y'] = df['Y'] + np.random.normal(size=len(df)) # without noise, the variance in Y|X, Z is zero, and mcmc fails.

In [0]:
cdf = CausalDataFrame(df)
cdf.causal.do(x={'A': 1}, 
              variable_types={'A': 'b', 'C': 'c', 'Y': 'c'}, 
              outcome='Y',
              method='mcmc', 
              common_causes=['C'],
              keep_original_treatment=True,
              proceed_when_unidentifiable=True).groupby('A').mean().plot(y='Y', kind='bar')

In [0]:
cdf = CausalDataFrame(df)

In [0]:
cdf_1 = cdf.causal.do(x={'A': 1}, 
              variable_types={'A': 'b', 'C': 'c', 'Y': 'b', 'S': 'b'}, 
              outcome='Y',
              method='mcmc', 
              dot_graph=data['dot_graph'],
              proceed_when_unidentifiable=True)

cdf_0 = cdf.causal.do(x={'A': 0}, 
              variable_types={'A': 'b', 'C': 'c', 'Y': 'b', 'S': 'b'}, 
              outcome='Y',
              method='mcmc', 
              dot_graph=data['dot_graph'],
              proceed_when_unidentifiable=True,
              use_previous_sampler=True)

In [0]:
cdf_0

In [0]:
cdf_1

In [0]:
(cdf_1['Y'] - cdf_0['Y']).mean()

In [0]:
1.96*(cdf_1['Y'] - cdf_0['Y']).std() / np.sqrt(len(cdf))

In [0]:
model = OLS(df['Y'], df[['C', 'A']])
result = model.fit()
result.summary()

In [0]:
cdf_do = cdf.causal.do(x={'A': 0}, 
              variable_types={'A': 'b', 'C': 'c', 'Y': 'c'}, 
              outcome='Y',
              method='mcmc', 
              common_causes=['C'],
              keep_original_treatment=True,
              proceed_when_unidentifiable=True).groupby('A').mean().plot(y='Y', kind='bar')

In [0]:
cdf_do

## Hands-on exercise: NHEFS data

In [0]:
nhefs = pd.ExcelFile('https://www.dropbox.com/s/nchp1pezska7bim/nhefs.xlsx?dl=1').parse('2017')
nhefs.columns

In [0]:
g = '''
graph [
directed 1
node[id "qsmk" label "qsmk"]
node[id "wt82_71" label "wt82_71"]
node[id "age" label "age"]
node[id "diabetes" label "diabetes"]
node[id "exercise" label "exercise"]
edge[source "qsmk" target "wt82_71" weight 1]
edge[source "qsmk" target "exercise" weight 1]
edge[source "wt82_71" target "exercise" weight 1]
edge[source "age" target "qsmk" weight 1]
edge[source "age" target "wt82_71" weight 1]
edge[source "diabetes" target "qsmk" weight 1]
edge[source "diabetes" target "wt82_71" weight 1]
]
'''

d = '''
digraph {
qsmk -> wt82_71;
qsmk -> exercise;
age -> qsmk;
age -> wt82_71;
diabetes -> qsmk;
diabetes -> wt82_71;
wt82_71 -> exercise;
U[label="Unobserved Confounders"];
U -> qsmk;
U -> wt82_71;
}
'''

selected = ['age', 'diabetes', 'qsmk', 'wt82_71', 'exercise']

data = {}
data['df'] = nhefs[selected]
data['treatment_name'] = 'qsmk'
data['outcome_name'] = 'wt82_71'
data['gml_graph'] = g.replace('\n', '')
data['dot_graph'] = d.replace('\n', '')

model = CausalModel(
    data=data["df"],
    treatment=data["treatment_name"],
    outcome=data["outcome_name"],
    graph=data["gml_graph"])

model.view_model()
display(Image(filename="causal_model.png"))

In [0]:
identified_estimand = model.identify_effect()
print(identified_estimand)

estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression",
    test_significance=True)

print(estimate)
print("Causal Estimate is " + str(estimate.value))

In [0]:
nhefs[selected].isnull().values.any()

In [0]:
from sklearn.preprocessing import Imputer

nhefs_imputed = nhefs
for i in selected:
  imputer = Imputer(missing_values="NaN", strategy="mean")
  nhefs_imputed[i] = imputer.fit_transform(nhefs[[i]]) 

data = {}
data['df'] = nhefs_imputed
data['treatment_name'] = 'qsmk'
data['outcome_name'] = 'wt82_71'
data['gml_graph'] = g.replace('\n', '')
data['dot_graph'] = d.replace('\n', '')

model = CausalModel(
    data=data["df"],
    treatment=data["treatment_name"],
    outcome=data["outcome_name"],
    graph=data["gml_graph"])

model.view_model()
display(Image(filename="causal_model.png"))

identified_estimand = model.identify_effect()
print(identified_estimand)

estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression",
    test_significance=True)

print(estimate)
print("Causal Estimate is " + str(estimate.value))